<a href="https://colab.research.google.com/github/NuttakitDW/DeepWork_216/blob/master/M_216.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
import xgboost as xgb

#import data
data = pd.read_csv('https://raw.githubusercontent.com/NuttakitDW/DeepWork_216/master/train_data.csv')
test_X = pd.read_csv('https://raw.githubusercontent.com/NuttakitDW/DeepWork_216/master/test_data.csv')

#prepare variable
y = data.income
X = data.drop(['income', 'id'], axis=1)
feature = ['ocp_cd', 'age', 'cc_mean',
           'cc_cnt_mean', 'kp_sum', 'kp_cnt_mean', 'overall']
X = X[feature]

#seperate column by type
categorical_cols = [cname for cname in ['ocp_cd', 'age']]
numerical_cols = [cname for cname in X if cname not in categorical_cols]

# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='constant')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

#create model
last_model = xgb.XGBRegressor(objective="reg:linear", random_state=42, 
                             learning_rate=0.01, gamma=0.3, max_depth=10, 
                             colsample_bytree=1, subsample=0.7, reg_alpha=0, reg_lambda=3,
                             min_child_weight=7, n_estimators=250)
#create pipeline
last_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', last_model)
                             ])

#training model
last_pipeline.fit(X, y)

#prediction
preds = last_pipeline.predict(test_X)

/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


[09:40:33] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [5]:
output = pd.DataFrame({'id': test_X['id'],
                       'income': preds})
output

,id,income
0,50001,30448.814453
1,50002,21446.480469
2,50003,31982.712891
3,50004,25830.621094
4,50005,31449.658203
...,...,...
14995,64996,21864.455078
14996,64997,26258.564453
14997,64998,25106.726562
14998,64999,41780.644531
